# Verifying MoCoDAD Pose Timing

The MoCoDAD paper does not provide latencies with pose estimation included.

The open source implementation (see paper for citation) includes a YOLO3-spp implementation so this notebook tests that model on just one HR-Avenue scene to FPS.


In [ ]:
import os
from src.sortalg import Sort
import torch
import cv2
from ultralytics import YOLO
from tqdm import tqdm

model = YOLO('yolov3-sppu', task='pose')

video_dir = '../Avenue Dataset/testing_videos'

with torch.no_grad():
        for video_name in os.listdir(video_dir):
            
            # only run for frames directories not loose files
            if "_frames" not in video_name:
                continue

            print(f"Processing video: {video_name}")
            frame_dir = os.path.join(video_dir, video_name)
            if not os.path.isdir(frame_dir):
                continue

            # initialise a new tracker at each video (prevent mistaken linking)
            tracker = Sort(max_age=10)
            frame_files = sorted(os.listdir(frame_dir))

            # process each file individually
            for frame_idx, frame_file in tqdm(enumerate(frame_files), total=len(frame_files)):
                # read the image
                frame_path = os.path.join(frame_dir, frame_file)
                frame = cv2.imread(frame_path)
                if frame is None:
                    continue

                
                # make predictions
                pred = model.predict(source=frame, save=False, verbose=False)